In [1]:
from utils.df_handle import *
from math import ceil

Default bq project:  spatial-vision-343005.biteam


In [2]:
import requests

In [3]:
from vars import vars

In [4]:
base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'

headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}

extra_url=f'raw/data-follow?&date_start=2022-03-23&limit=500'

In [5]:
r = requests.get(base_url+extra_url, headers=headers)

data = r.json()['data']

# df1=pd.DataFrame.from_records(data)

In [6]:
# df1.shape

In [7]:
data

[{'customer_code': 'M1201138',
  'customer_phone': '',
  'customer_name': 'NT Minh Hải - Quận 12 - HCM',
  'follow_name': 'Nguyễn Ngọc Trâm',
  'follow_phone': '0703940905',
  'gender': 1,
  'birthday': '1998-09-07',
  'follow_address': '88A Nguyễn Ảnh Thủ, Phường Hiệp Thành, Quận 12',
  'customer_address': 'KP2 Nguyễn ảnh Thủ ,, Phường Hiệp Thành, Quận 12, Thành phố Hồ Chí Minh',
  'updated_at': '2022-09-26 16:42:56',
  'customer_role_name': 'Người phụ trách đặt hàng',
  'office_code': 'MR0001',
  'office_name': 'CÔNG TY CỔ PHẦN DƯỢC PHA NAM',
  'user_name': 'Trần Tuấn Anh',
  'user_code': 'MR2547',
  'status': 1},
 {'customer_code': 'M1301041',
  'customer_phone': '',
  'customer_name': 'NT Mai Uyên - Gò Vấp - HCM',
  'follow_name': 'Trần Thị Kim Ngọc',
  'follow_phone': '0986818413',
  'gender': 1,
  'birthday': '1984-09-12',
  'follow_address': '122 Lê Đức Thọ, Phường 06 Quận Gò Vấp, Thành phố Hồ Chí Minh',
  'customer_address': '122 Lê Đức Thọ, Phường 06, Quận Gò Vấp, Thành phố Hồ

In [6]:
def insert_first_time():
    #*--------------*
    #get pages
    base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'

    headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}

    extra_url=f'raw/data-follow?&limit=1'
    
    print("url is: ",base_url+extra_url)
    
    r = requests.get(base_url+extra_url, headers=headers)
    
    r.json()
    
    data = r.json()['data']
    
    print("total output: " ,r.json()['total'])
    
    print("do dai list api: ",len(data))
    
    maxpage = ceil(r.json()['total']/500)
    
    pages = [x for x in range(1, maxpage+1, 1)]
    
    limit=500
    #end get pages
    #*--------------*
    for page in pages:
        base_url = 'https://eoffice.merapgroup.com/eoffice/api/api/'
        headers = {"Authorization": f"Bearer {vars['name_eoffice']}", "Content-Type":"application/json"}
        extra_url=f'raw/data-follow?limit={limit}&page={page}'
        r = requests.get(base_url+extra_url, headers=headers)
        r.json()
        data = r.json()['data']
        # len(data)
        df1=pd.DataFrame.from_records(data)
        df1.columns = ['makhdms', 'customer_phone', 'customer_name', 'follow_name',
            'follow_phone', 'gender', 'birthday', 'follow_address',
            'customer_address', 'updated_at', 'customer_role_name', 'office_code',
            'office_name', 'user_name', 'user_code', 'status']
        lst = ['makhdms', 'customer_phone', 'customer_name', 'follow_name',
            'follow_phone', 'follow_address',
            'customer_address','status','updated_at']
        df1=df1[lst]
        vc(df1, 'status')
        dk = df1['status'] != 0
        df1 = df1[dk].copy()
        # drop_cols(df1, 'status')
        print("page inserted: ", page)
        execute_values_upsert(df1, 'f_eoffice_cus_infor', pk=['follow_phone'])

In [44]:
# insert_first_time([1,2,3,4,5,6,7])